In [1]:
## json -> txt 변환
import os
import json
# 작업 결과물을 저장할 디렉토리 경로
save_path = r"D:\150.버스 승객 승하차 영상\150.버스 승객 승하차 영상\01.데이터\1.Training\TL\General\txtA"

# JSON 파일을 읽어 TXT 파일로 변환하는 함수
def convert_json_to_txt(json_file_path, save_path):
    with open(json_file_path, "r") as file:
        json_data = json.load(file)

    WIDTH = float(json_data['info']['width'])
    HEIGHT = float(json_data['info']['height'])
                                        ## train은 laydown, valid는 lay down
    CLASS_NAMES = ["stand", "walk", "run", "sit down", "lay down", "grab handle", "kick", "punch"]

    output_file_path = os.path.join(save_path, os.path.splitext(os.path.basename(json_file_path))[0] + ".txt")

    with open(output_file_path, "a") as new_file:
        annotations = json_data["annotations"]
        for i, data in enumerate(annotations):
            class_name = data['action']
            class_id = CLASS_NAMES.index(class_name)
            bbox = data['bbox']
            x_left_top = bbox[0]
            y_left_top = bbox[1]
            x_right_bottom = bbox[2]
            y_right_bottom = bbox[3]
            width = x_right_bottom - x_left_top
            height = y_right_bottom - y_left_top
            normalize_width = round(width / WIDTH, 6)
            normalize_height = round(height / HEIGHT, 6)
            ctr_x = x_left_top + width / 2
            ctr_y = y_left_top + height / 2
            normalize_ctr_x = round(ctr_x / WIDTH, 6)
            normalize_ctr_y = round(ctr_y / HEIGHT, 6)

            new_data = "{} {} {} {} {}\n".format(str(class_id), str(normalize_ctr_x), str(normalize_ctr_y), str(normalize_width), str(normalize_height))
            new_file.write(new_data)

# 주어진 디렉토리에서 모든 JSON 파일을 찾고 변환 작업을 수행하는 함수
def process_directory(directory_path, save_path):
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".json"):
                json_file_path = os.path.join(root, file)
                convert_json_to_txt(json_file_path, save_path)

# 변환할 디렉토리 경로
directory_path = r"D:\150.버스 승객 승하차 영상\150.버스 승객 승하차 영상\01.데이터\1.Training\TL\General\A"

# 디렉토리 내의 모든 JSON 파일을 TXT로 변환
process_directory(directory_path, save_path)

In [9]:
import os
import shutil
import random

def split_data(image_folder, label_folder, train_ratio=0.8):
    # 이미지 파일 및 라벨링 파일 목록을 가져옴
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.json') or f.endswith('.txt')]
    
      # 데이터 쌍을 형성
    image_label_pairs = []
    for image_file in image_files:
        image_name, _ = os.path.splitext(image_file)
        label_file = image_name + '.json' if image_name + '.json' in label_files else image_name + '.txt'
        if label_file in label_files:
            image_label_pairs.append((image_file, label_file))
    
    # 데이터 쌍을 무작위로 섞음
    random.shuffle(image_label_pairs)
    
    # 데이터 분할 인덱스 계산
    train_count = int(len(image_label_pairs) * train_ratio)
    
    # 학습 데이터와 테스트 데이터 분할
    train_data = image_label_pairs[:train_count]
    test_data = image_label_pairs[train_count:]
    
    return train_data, test_data

def move_data(data, source_image_folder, source_label_folder, destination_image_folder, destination_label_folder):
    for image_file, label_file in data:
        # 이미지 파일 이동
        source_image_path = os.path.join(source_image_folder, image_file)
        destination_image_path = os.path.join(destination_image_folder, image_file)
        shutil.move(source_image_path, destination_image_path)
        
        # 라벨링 파일 이동
        source_label_path = os.path.join(source_label_folder, label_file)
        destination_label_path = os.path.join(destination_label_folder, label_file)
        shutil.move(source_label_path, destination_label_path)

# 소스 이미지와 라벨링 폴더 경로 설정
source_image_folder = r"D:\150.버스 승객 승하차 영상\이미지"
source_label_folder = r"D:\150.버스 승객 승하차 영상\라벨\txtA"

# 학습 및 테스트 이미지 폴더 설정
train_image_folder = r"D:\trainimage"
train_label_folder = r"D:\trainlabel"
test_image_folder = r"D:\testimage"
test_label_folder = r"D:\testlabel"

# 데이터 분할
train_data, test_data = split_data(source_image_folder, source_label_folder)

# 데이터 이동
move_data(train_data, source_image_folder, source_label_folder, train_image_folder, train_label_folder)
move_data(test_data, source_image_folder, source_label_folder, test_image_folder, test_label_folder)


In [4]:
import os
import shutil
import random

def split_data(image_folder, label_folder, train_ratio=0.8):
    # 이미지 파일 및 라벨링 파일 목록을 가져옴
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    label_files = [f for f in os.listdir(label_folder) if f.endswith('.json') or f.endswith('.txt')]
    
    # 데이터 쌍을 형성
    image_label_pairs = []
    for image_file in image_files:
        image_name, _ = os.path.splitext(image_file)
        label_file = image_name + '.json' if image_name + '.json' in label_files else image_name + '.txt'
        if label_file in label_files:
            image_label_pairs.append((image_file, label_file))
    
    # 데이터 쌍을 무작위로 섞음
    random.shuffle(image_label_pairs)
    
    # 데이터 분할 인덱스 계산
    train_count = int(len(image_label_pairs) * train_ratio)
    
    # 학습 데이터와 테스트 데이터 분할
    train_data = image_label_pairs[:train_count]
    test_data = image_label_pairs[train_count:]
    
    return train_data, test_data

def copy_data(data, source_image_folder, source_label_folder, destination_image_folder, destination_label_folder):
    for image_file, label_file in data:
        # 이미지 파일 복사
        source_image_path = os.path.join(source_image_folder, image_file)
        destination_image_path = os.path.join(destination_image_folder, image_file)
        shutil.copyfile(source_image_path, destination_image_path)
        
        # 라벨링 파일 복사
        source_label_path = os.path.join(source_label_folder, label_file)
        destination_label_path = os.path.join(destination_label_folder, label_file)
        shutil.copyfile(source_label_path, destination_label_path)

# 소스 이미지와 라벨링 폴더 경로 설정
source_image_folder = r"D:\150.버스 승객 승하차 영상\150.버스 승객 승하차 영상\01.데이터\1.Training\원천데이터_231214_add\150.버스 승객 승하차 영상\01.데이터\1.Training\원천데이터_231214_add\General\A"
source_label_folder = r"D:\150.버스 승객 승하차 영상\150.버스 승객 승하차 영상\01.데이터\1.Training\TL\General\txtA"

# 학습 및 테스트 이미지 폴더 설정
train_image_folder = r"D:\trainimage"
train_label_folder = r"D:\trainlabel"
test_image_folder = r"D:\testimage"
test_label_folder = r"D:\testlabel"

# 데이터 분할
train_data, test_data = split_data(source_image_folder, source_label_folder)

# 데이터 복사
copy_data(train_data, source_image_folder, source_label_folder, train_image_folder, train_label_folder)
copy_data(test_data, source_image_folder, source_label_folder, test_image_folder, test_label_folder)


In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [1]:
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 경로 설정
src_image_folder = r"D:\150.버스 승객 승하차 영상\01.데이터\1.Training\원천데이터_231214_add\General\A"
src_label_folder = r"D:\150.버스 승객 승하차 영상\라벨"
train_image_folder = r"D:\trainimage"
train_label_folder = r"D:\trainlabel"
valid_image_folder = r"D:\testimage"
valid_label_folder = r"D:\testlabel"

# 파일 목록 수집 함수
def collect_files(folder_path, extensions):
    return [os.path.join(root, file) for root, dirs, files in os.walk(folder_path) for file in files if file.endswith(extensions)]

# 파일 이동 함수
def move_file(src_dest_tuple):
    src, dest = src_dest_tuple
    shutil.move(src, dest)
    return 1

# 이미지와 라벨 파일 매칭
image_files = collect_files(src_image_folder, ('.jpg', '.png'))
label_files = collect_files(src_label_folder, '.txt')

image_labels_pairs = {os.path.splitext(os.path.basename(image))[0]: (image, '') for image in image_files}
for label in label_files:
    base_name = os.path.splitext(os.path.basename(label))[0]
    if base_name in image_labels_pairs:
        image_labels_pairs[base_name] = (image_labels_pairs[base_name][0], label)

# 파일 분할 및 이동
paired_files = list(image_labels_pairs.values())
random.shuffle(paired_files)
cutoff = int(0.8 * len(paired_files))

train_files = paired_files[:cutoff]
valid_files = paired_files[cutoff:]

def move_files(file_paths, image_dest_folder, label_dest_folder, label):
    image_dest_pairs = [(path[0], os.path.join(image_dest_folder, os.path.basename(path[0]))) for path in file_paths if path[0]]
    label_dest_pairs = [(path[1], os.path.join(label_dest_folder, os.path.basename(path[1]))) for path in file_paths if path[1]]
    
    with ThreadPoolExecutor(max_workers=8) as executor:
        list(tqdm(executor.map(move_file, image_dest_pairs), total=len(image_dest_pairs), desc=f"{label} Images"))
        list(tqdm(executor.map(move_file, label_dest_pairs), total=len(label_dest_pairs), desc=f"{label} Labels"))
    
    return len(label_dest_pairs)

# Train 파일 이동 및 처리 수 계산
train_label_count = move_files(train_files, train_image_folder, train_label_folder, 'Train')

# Validation 파일 이동 및 처리 수 계산
valid_label_count = move_files(valid_files, valid_image_folder, valid_label_folder, 'Valid')

print(f"훈련 세트에 이동된 JSON 파일 수: {train_label_count}개")
print(f"검증 세트에 이동된 JSON 파일 수: {valid_label_count}개")

Valid Labels: 100%|███████████████████████████████████████████████████████████| 117544/117544 [14:15<00:00, 137.47it/s]

훈련 세트에 이동된 JSON 파일 수: 470176개
검증 세트에 이동된 JSON 파일 수: 117544개
